In [1]:
from __future__ import print_function

import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import accuracy_score

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Bi-LSTM Conditional Random Field for NER (`State of the art`)

- [PyTorch - Bi-LSTM + CRF](http://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html)
- [Michael Collins CRF](http://www.cs.columbia.edu/~mcollins/crf.pdf)
- [TensorFlow - Bi-LSTM + CRF with character embeddings for NER](https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html)

- Let $\mathbf{x} = (x^1,...,x^M)$ denote a sequence of words and $\mathbf{y} = (y^1,...,y^M)$ denote the corresponding sequence of part-of-speech tags.
- $M$ is the length of a sequence and $\mathbf{x}$ is encoded using $D$ words. So there are $D^M$ possible length-$M$ sequences of $\mathbf{x}$
- There are $L$ possible labels for each $y^i$ so there are $L^M$ possible length-$M$ sequences of $\mathbf{y}$
- Training set of $N$ training examples: $S = \{(\mathbf{x}_i, \mathbf{y}_i)\}_{i=1}^N$
- Goal is to learn mapping from $\mathbf{x}$ to $\mathbf{y}$ by fitting a first order sequential Conditional Random Field (CRF) to $S$. A CRF is a `log-linear` conditional probabilistic model.

---
*$1^{st}$ Order Hidden Markov Model (*For understanding CRF*)*
- Generative model $P(x, y)$, i.e. joint model over $x$ and $y$
- Notation
    - $P(x^i|y^i)$: Probability of state $y^i$ generating $x^i$
    - $P(y^{i+1}|y^i)$: Probability of state $y^i$ transitioning to $y^{i+1}$
    - $P(y^1|y^0)$: $y^0$ is defined to be the start state
    - $P(END|y^M)$: Prior probability of $y^M$ being the final state (Not always used)

$$P(x, y) = P(END|y^M)\prod_{i=1}^M{P(y^i|y^{i-1})}\prod_{i=1}^M{P(x^i|y^i})$$

- In matrix notation (where $A$ denote `transition` probabilities and $O$ denote `emission` or `observation` probabilities)
$$P(x, y) = A_{END, y^M}\prod_{i=1}^M{A_{y^i, y^{i-1}}O_{y^i, x^i}}$$

---
Log-Linear $1^{st}$ Order Sequential CRF Model

$$P(y|x) = \frac{1}{Z(x)}\exp\left\{\sum_{i=1}^M(A_{y^i, y^{i-1}}O_{y^i, x^i})\right\} \equiv \frac{1}{Z(x)}\exp\{F(y,x)\}$$

- $Z(x) = \sum_{y'}\exp{F(y', x)}$ is `Partition function` which sums over the scores of all possible $y'$ and acts as a normalizer so that $P(y|x)$ yields a valid probability
- $F(y,x) \equiv \sum_{i=1}^M(A_{y^i, y^{i-1}}O_{y^i, x^i})$ is `scoring` function or `joint discriminant` function
- Example: $x$ = "Dog Barks", $y$ = (N, V)

$$P(N,V|\text{"Dog Barks"}) = \frac{1}{Z(x)}\exp\left\{A_{N,Start} + O_{N, Dog} + A_{V,N} + O_{V, Barks}\right\}$$

---
- Training via Gradient Descent and Dynamic Programming
    - Gradienet descent to minimize the negative log loss over a training set. The gradient of log partition i.e. $\text{log }Z(x)$ is equal to the sum of the expected value of each feature under the distribution induced by the current model $P(x, y)$. `Forward-Backward` algorithm is used to compute the expected value in the formulation of gradient of log partition
    
$$\underset{\theta}{argmin}\sum_{i=1}^N - log(P(y_i|x_i)) = \underset{\theta}{argmin}\sum_{i=1}^N -F(y_i, x_i) + log(Z(x_i))$$

---

- Prediction is made by using `Viterbi` Algorithm (one line in `TensorFlow`)
    - $\underset{y}{argmax}\text{ }P(y|x) = \underset{y}{argmax}\text{ }log(P(y|x)) =\underset{y}{argmax}\text{ }F(y,x)$
    - Viterbi algorithm does not compute $P(y|x)$, it just maximizes $F(y,x)$ so need to compute the partition function $Z(x)$
    - Viterbi algorithm finds an entire sequence of states such that the sum of transition scores is maximized
    
---

- Computing `Partition Function`
    - Notation: Let matrix $G^i(b, a) = \exp\{A_{b, a}+O_{b, x^i}\}$ encode the unnormalized probability of transitioning from $y^{i-1} = a$ to $y^i=b$ in the $i$-th token of the sequence. 

$$Z(x) = \sum_{y'}\prod_{i=1}^MG^i(y^{'i}, y^{'i-1})$$


- Use Viterbi Algorithm to compute `Partition Function`

In [441]:
# Helper functions for BiLSTM_CRF
def to_scalar(var):
    return var.view(-1).data.tolist()[0]

def argmax(vec):
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)

# Compute log sum exp in a numerically stable for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

# Helper functions for preparing sequences for BiLSTM_CRF
def prepare_sequence(seq, to_idx, train=True):
    indices = [to_idx[w] for w in seq]
    if cuda:
        tensor = torch.LongTensor(indices).cuda()
    else:
        tensor = torch.LongTensor(indices)
        
    if train:
        return autograd.Variable(tensor, requires_grad=False)
    else:
        return autograd.Variable(tensor, requires_grad=False, volatile=True) # Solves GPU running out of memory issue

In [442]:
# Create model

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters. Entry i,j is the score of
        # transitioning to i from j
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        
        # Enforce the constraint: never transition to start tag and never 
        # transition from stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000.0
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000.0

        self.hidden = self.init_hidden()

    def init_hidden(self):
        if cuda:
            return (autograd.Variable(torch.randn(2, 1, self.hidden_dim // 2)).cuda(), 
                    autograd.Variable(torch.randn(2, 1, self.hidden_dim // 2)).cuda())
        else:
            return (autograd.Variable(torch.randn(2, 1, self.hidden_dim // 2)), 
                    autograd.Variable(torch.randn(2, 1, self.hidden_dim // 2)))

    def _forward_alg(self, feats):
        """Forward algorithm for computing the partition function"""
        init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.0)
                    
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.0

        # Wrap in a variable so that we will get automatic backprop
        if cuda:
            forward_var = autograd.Variable(init_alphas).cuda()
        else:
            forward_var = autograd.Variable(init_alphas)

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward variables at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        """Returns LSTM output for a given sequence"""
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        """Returns the score of a provided tag sequence"""
        if cuda:
            score = autograd.Variable(torch.Tensor([0])).cuda()
        else:
            score = autograd.Variable(torch.Tensor([0]))
            
        tags = torch.cat([torch.LongTensor([self.tag_to_ix[START_TAG]]), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        """Uses Viterbi algorithm to compute score and best tag sequence"""
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.0)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        if cuda:
            forward_var = autograd.Variable(init_vvars).cuda()
        else:
            forward_var = autograd.Variable(init_vvars)
        
        for feat in feats:
            bptrs_t = [] # holds the backpointers for this step
            viterbivars_t = [] # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id])
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def forward(self, sentence):
        """Returns score and tag sequence"""
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq
    
    def neg_log_likelihood(self, sentence, tags):
        """Negative Log Likelihood"""
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
      
def train(model, optimizer, train_data):
    """
    train_data -- list of tuples, e.g. [(['dog', 'ate'], ['NN', 'V']), (...), ...]
    """
    for sentence, tags in train_data:
        
        # Step 1: Clear gradients as Pytorch accumulates gradients
        model.zero_grad()
        
        # Step 2: Prepare inputs
        sentence_in = prepare_sequence(sentence, word_to_idx)
        targets = torch.LongTensor([tag_to_idx[tag] for tag in tags])
        
        # Step 3: Run forward pass and compute loss
        neg_log_likelihood = model.neg_log_likelihood(sentence_in, targets)
        
        # Step 4: Compute gradients, optimize and update the gradients
        neg_log_likelihood.backward()
        optimizer.step()
          
def evaluate(model, data):
    model.eval()
    
    correct = 0
    for sentence, tags in data:
        
        # Step 1: Prepare inputs
        sentence_in = prepare_sequence(sentence, word_to_idx, train=False)
        targets = torch.LongTensor([tag_to_idx[tag] for tag in tags])
        
        # Step 2: Forward pass
        pred_score, pred_tags = model(sentence_in)
                            
        # Step 3: Compute accuracy
        correct += torch.equal(torch.LongTensor(pred_tags), targets)
        
    return correct/float(len(data))

# Keep only a single checkpoint, the best over test accuracy.
def save_checkpoint(state, is_best, filename=None):
    """Save checkpoint if a new best is achieved"""
    if is_best:
        print ("### Saving New Best Model Weights ###")
        torch.save(state, filename)  # save checkpoint
    else:
        print ("Validation Accuracy did not improve!")

In [445]:
# Hyperparameters and data preparation
START_TAG = '<START>'
STOP_TAG = '<STOP>'
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
num_epochs = 5

learning_rate = 0.01
best_accuracy = torch.FloatTensor([0])
start_epoch = 0

# Path to saved model weights (as hdf5)
resume_weights = './bi-lstm-crf-ner/checkpoint.pth.tar'

# GPU?
cuda = torch.cuda.is_available()

# Set GPU to use
torch.cuda.set_device(2)

# Seed for reproducibility
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)

# Data
training_data = [
    ('the wall street journal reported today that apple corporation made money'.split(),
     'B I I I O O O B I O O'.split()),
    ('georgia tech is a university in georgia'.split(),
     'B I O O O O B'.split())
]

testing_data = [
    ('california tech is a university in california'.split(), # 'caltech' actually
     'B I O O O O B'.split())
]

word_to_idx = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
            
for sentence, tags in testing_data:
    for word in sentence:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
            
tag_to_idx = {'B': 0, 'I': 1, 'O': 2, START_TAG: 3, STOP_TAG: 4}

print('Vocab size: ', len(word_to_idx))

Vocab size:  18


In [446]:
# Model and optimizer
model = BiLSTM_CRF(len(word_to_idx), tag_to_idx, EMBEDDING_DIM, HIDDEN_DIM)

if cuda:
    model.cuda()
    
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# If best model weights exist then load it
if os.path.isfile(resume_weights):
    print('Loading checkpoint: "{}" ...'.format(resume_weights))
    
    # Load weights
    checkpoint = torch.load(resume_weights)
    
    start_epoch = checkpoint['epoch']
    best_accuracy = checkpoint['best_accuracy']
    model.load_state_dict(checkpoint['state_dict'])
    print('Loaded checkpoint "{}" (trained for {} epochs)'.format(resume_weights, start_epoch))

Loading checkpoint: "./bi-lstm-crf-ner/checkpoint.pth.tar" ...
Loaded checkpoint "./bi-lstm-crf-ner/checkpoint.pth.tar" (trained for 319 epochs)


In [447]:
# Train the model
start = datetime.now()
for epoch in range(num_epochs):
    train(model, optimizer, training_data)
    acc = evaluate(model, testing_data)
    tr_acc = evaluate(model, training_data)
    print('Epoch-{} Test Set: Accuracy: {:.2f}'.format(epoch, acc))
    print('Epoch-{} Train Set: Accuracy: {:.2f}'.format(epoch, tr_acc))
        
    acc = torch.FloatTensor([acc])
    
    # Get bool not ByteTensor
    is_best = bool(acc.numpy() > best_accuracy.numpy())
    
    # Get greater tensor to keep track of best_accuracy
    best_accuracy = torch.FloatTensor(max(acc.numpy(), best_accuracy.numpy()))
    
    # Save checkpoint
    save_checkpoint({'epoch': start_epoch + epoch + 1,
                     'state_dict': model.state_dict(),
                     'optimizer_state_dict': optimizer.state_dict(),
                     'best_accuracy': best_accuracy,}, is_best, filename=resume_weights)
    
end = datetime.now()
elapsed = end - start
print('Training Time: ', elapsed)

Epoch-0 Test Set: Accuracy: 0.00
Epoch-0 Train Set: Accuracy: 1.00
Validation Accuracy did not improve!
Epoch-1 Test Set: Accuracy: 0.00
Epoch-1 Train Set: Accuracy: 1.00
Validation Accuracy did not improve!
Epoch-2 Test Set: Accuracy: 0.00
Epoch-2 Train Set: Accuracy: 1.00
Validation Accuracy did not improve!
Epoch-3 Test Set: Accuracy: 0.00
Epoch-3 Train Set: Accuracy: 1.00
Validation Accuracy did not improve!
Epoch-4 Test Set: Accuracy: 0.00
Epoch-4 Train Set: Accuracy: 1.00
Validation Accuracy did not improve!
Training Time:  0:00:00.693733


In [448]:
# Prediction
test_sent = prepare_sequence(testing_data[0][0], word_to_idx, train=False)
test_tags = torch.LongTensor([tag_to_idx[tag] for tag in testing_data[0][1]])
zip(model(test_sent)[1], test_tags)

[(1, 0), (1, 1), (2, 2), (2, 2), (2, 2), (2, 2), (1, 0)]

### Algorithms

**Viterbi Algorithm**

**Viterbi Algorithm Explanation**
- Dynamic Programming algorithm
- Used to find the most likely sequence of hidden states that results in a sequence of observed events
- HMM - The state is not directly visible but the observations dependent on the state are visible


- **Input**
    - State space $S=\{s_1, s_2, ..., s_N\}$
    - Observation space $O=\{o_1, o_2, ..., o_K\}$
    - Transition matrix $\underset{N \times N}A$ such that $A_{i,j}$ stores the transition probability of transiting from state $s_i$ to $s_j$
    - Emission matrix $\underset{N \times K}B$ such that $B_{i,j}$ stores the probability of observing $o_j$ from state $s_i$
    - An array of initial probabilities $\pi$ of size $N$ such that $\pi_i$ stores the probability of state $s_i$ at time $t=1$
    - Sequence of observations $y_1, y_2, ..., y_T$
- **Output**
    - Most likely hidden state sequence $X = \{x_1, x_2, ..., x_T\}$

In [409]:
import numpy as np

# Initialize the model parameters

# Transition matrix A(i,j): Probability of transitioning from state i to state j
A = np.array([[0.6, 0.2, 0.2],
              [0.5, 0.3, 0.2],
              [0.4, 0.1, 0.5]])

# Array of initial probabilities
pi = np.array([0.5, 0.2, 0.3]) 

# Emission matrix O(i, j): Probability of observing Oj from state Si
O = np.array([[0.7, 0.1, 0.2],
              [0.1, 0.6, 0.3],
              [0.3, 0.3, 0.4]])

# State space
states = UP, DOWN, UNCHANGED = 0, 1, 2

observations = [UP, UP, DOWN]

# Viterbi algorithm keeps a store of 2 components: (1) Best score to reach
# a state at a given time; (2) Last step of the path to get there

alpha = np.zeros((len(observations), len(states))) # time steps x states
alpha[:,:] = float('-inf') # Initialized to -inf to denote that values are not set yet
backpointers = np.zeros((len(observations), len(states)), dtype='int')

# Base case for recursion sets the starting state probabilities based on phi and generating
# the observations
alpha[0, :] = pi * O[:, UP]
print(alpha)

# Recursive step: Maximize over incoming transitions reusing the best incoming score computed 
# above

# time step 1
for t1 in states:
    for t0 in states:
        score = alpha[0, t0] * A[t0, t1] * O[t1, UP]
        if score > alpha[1, t1]:
            alpha[1, t1] = score
            backpointers[1, t1] = t0
            
print(alpha)

# time step 2
for t2 in states:
    for t1 in states:
        score = alpha[1, t1] * A[t1, t2] * O[t2, DOWN]
        if score > alpha[2, t2]:
            alpha[2, t2] = score
            backpointers[2, t2] = t1
            
print(alpha)

def viterbi(params, observations):
    pi, A, O = params
    M = len(observations)
    S = pi.shape[0]
    
    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf') # Initialized to -inf
    backpointers = np.zeros((M, S), dtype='int')
    
    # base case
    alpha[0, :] = pi * O[:,observations[0]]
    
    # Recursive case
    for t in range(1, M):
        for s2 in range(S):
            for s1 in range(S):
                score = alpha[t-1, s1] * A[s1, s2] * O[s2, observations[t]]
                if score > alpha[t, s2]:
                    alpha[t, s2] = score
                    backpointers[t, s2] = s1
    
    # Follow backpointers to resolve the state sequence
    ss = []
    ss.append(np.argmax(alpha[M-1,:]))
    for i in range(M-1, 0, -1):
        ss.append(backpointers[i, ss[-1]])
        
    return list(reversed(ss)), np.max(alpha[M-1,:])

viterbi((pi, A, O), [UP, UP, DOWN])

[[ 0.35  0.02  0.09]
 [ -inf  -inf  -inf]
 [ -inf  -inf  -inf]]
[[ 0.35   0.02   0.09 ]
 [ 0.147  0.007  0.021]
 [  -inf   -inf   -inf]]
[[ 0.35     0.02     0.09   ]
 [ 0.147    0.007    0.021  ]
 [ 0.00882  0.01764  0.00882]]


([0, 0, 1], 0.017639999999999999)

**Forward-Backward Algorithm**

- Find the probability of an observation sequence *under any hidden state sequence*. This allows HMM to function as language models, but also is key to unsupervised training and the central algorithm for training.
- The quantity to compute is $p(w) = \sum_{t} p(t,w)$ where $w$ are the observations (words) and $t$ are the states (tags).

$$p(w)  = \sum_{t_1} \sum_{t_2} \cdots \sum_{t_{N-1}} \sum_{t_N} p(t,w)$$

- Expand the HMM probability 

$$
p(w)  = \sum_{t_1} \sum_{t_2} \cdots \sum_{t_{N-1}} \sum_{t_N} p(t_1) p(w_1 | t_1) p(t_2 | t_1) p(w_2| t_2) \cdots p(t_{N-1} | t_{N-2}) p(w_{N-1}| t_{N-1}) p(t_{N} | t_{N-1}) p(w_{N}| t_{N})
$$

- Compare the full marginal probability $p(w)$ and the probability up to position $N-1$, finishing with tag $t_{N-1}$

$$p(w_1, w_2, \ldots, w_{N-1}, t_{N-1}) = \sum_{t_1} \sum_{t_2} \cdots \sum_{t_{N-1}} p(t_1) p(w_1 | t_1) p(t_2 | t_1) p(w_2| t_2) \cdots p(t_{N-1} | t_{N-2}) p(w_{N-1}| t_{N-1})
$$

- Express $p(w)$ more simply as

$$
p(w)  = \sum_{t_N} p(w_1, w_2, \ldots, w_{N-1}, t_{N-1}) p(t_{N} | t_{N-1}) p(w_{N}| t_{N})
$$

- Continue further by defining $p(w_1, w_2, \ldots, w_{N-1}, t_{N-1})$ in terms of $p(w_1, w_2, \ldots, w_{N-2}, t_{N-2})$ and so forth. (This is the same process used in the Viterbi algorithm, albeit swapping a max for a sum.)

- Store a matrix of partial marginals, $\alpha$ defined as follows

$$\alpha[i, t_i] = p(w_1, w_2, \ldots, w_i, t_i)$$

computed using the recursion

$$\alpha[i, t_i] = \sum_{t_{i-1}} \alpha[i-1, t_i] p(t_i | t_{i-1}) p(w_i| t_i)$$

and the base case for $i=1$,

$$\alpha[1, t_1] = p(t_1) p(w_1 | t_1)$$

- Iteratively compute the vector of alpha[1] values, then alpha[2] etc until the end of input

- Backward Algorithm: The same process but working from left to right rather than right to left. Backward Algorithm is used for unsupervized learning (not 100% sure as I just read on few slides)

In [410]:
def forward(params, observations):
    pi, A, O = params
    N = len(observations)
    S = pi.shape[0]
    
    alpha = np.zeros((N, S))
    
    # Base case
    alpha[0, :] = pi * O[:,observations[0]]
    
    # Recursive case
    for i in range(1, N):
        for s2 in range(S):
            for s1 in range(S):
                alpha[i, s2] += alpha[i-1, s1] * A[s1, s2] * O[s2, observations[i]]
    
    return (alpha, np.sum(alpha[N-1,:]))

def backward(params, observations):
    pi, A, O = params
    N = len(observations)
    S = pi.shape[0]
    
    beta = np.zeros((N, S))
    
    # Base case
    beta[N-1, :] = 1
    
    # Recursive case
    for i in range(N-2, -1, -1):
        for s1 in range(S):
            for s2 in range(S):
                beta[i, s1] += beta[i+1, s2] * A[s1, s2] * O[s2, observations[i+1]]
    
    return (beta, np.sum(pi * O[:, observations[0]] * beta[0,:]))


forward((pi, A, O), [UP, UP, DOWN]), backward((pi, A, O), [UP, UP, DOWN])

((array([[ 0.35    ,  0.02    ,  0.09    ],
         [ 0.1792  ,  0.0085  ,  0.0357  ],
         [ 0.012605,  0.025176,  0.016617]]), 0.054398000000000002),
 (array([[ 0.1216,  0.1077,  0.1076],
         [ 0.24  ,  0.29  ,  0.25  ],
         [ 1.    ,  1.    ,  1.    ]]), 0.054397999999999995))

## Scratch

In [326]:
st = '<START>'
tag_to_idx = {'<START>': 1, 'NN': 0, 'V': 2, '<STOP>':3}
a = torch.Tensor(1, len(tag_to_idx)).fill_(-1000.0)
print(a)
a[0][tag_to_idx[st]] = 0
a


-1000 -1000 -1000 -1000
[torch.FloatTensor of size 1x4]




-1000     0 -1000 -1000
[torch.FloatTensor of size 1x4]

In [83]:
# dtype = torch.cuda.FloatTensor if cuda else torch.FloatTensor
# indices = [word_to_idx[w] for w in testing_data[0][0]]
# tensor = torch.LongTensor(indices).type(dtype)
# print(type(tensor))
# E = nn.Embedding(len(word_to_idx), EMBEDDING_DIM).cuda()
# E(tensor)

In [84]:
if torch.cuda.is_available():
    # Create a LongTensor and transfers it to GPU as torch.cuda.LongTensor
    a = torch.LongTensor(10).fill_(3).cuda()
    print(type(a))
    b = a.cpu()
    # Transfer it to CPU, back to being a torch.LongTensor
    print(type(a))
    print(type(b))

<class 'torch.cuda.LongTensor'>
<class 'torch.cuda.LongTensor'>
<class 'torch.LongTensor'>


In [88]:
test_sent = prepare_sequence(testing_data[0][0], word_to_idx)
print(type(test_sent))
if cuda:
    test_sent.cuda()
    
print(type(test_sent))

<class 'torch.autograd.variable.Variable'>
<class 'torch.autograd.variable.Variable'>


In [102]:
# # TypeError fix
# TypeError: torch.index_select received an invalid combination of arguments - 
# got (torch.cuda.FloatTensor, int, torch.LongTensor), but expected 
# (torch.cuda.FloatTensor source, int dim, torch.cuda.LongTensor index)

indices = [word_to_idx[w] for w in testing_data[0][0]]
tensor = torch.LongTensor(indices).cuda()
V = autograd.Variable(tensor, requires_grad=False)
E = nn.Embedding(len(word_to_idx), EMBEDDING_DIM).cuda()
E(V), type(tensor), type(V), type(E(V))

(Variable containing:
  0.2793 -1.8519 -1.0544 -2.8030  1.6265
  0.8026 -0.3939  0.6067  1.5321  0.0224
  0.5590  0.7341  0.9777  1.1262  0.4591
 -0.4116 -0.6755 -0.2887  0.6900  0.3158
 -1.4657  1.2674  0.8226 -1.4745  0.5069
 -0.9811 -2.3167  0.3534  0.0263 -0.2052
  0.2793 -1.8519 -1.0544 -2.8030  1.6265
 [torch.cuda.FloatTensor of size 7x5 (GPU 2)],
 torch.cuda.LongTensor,
 torch.autograd.variable.Variable,
 torch.autograd.variable.Variable)

In [320]:
def prepare_sequence_old(seq, to_idx, train=True):
    """NOTE: This does not work well!"""
    dtype = torch.cuda.FloatTensor if cuda else torch.FloatTensor
    indices = [to_idx[w] for w in seq]
    tensor = torch.LongTensor(indices)
    if train:
        return autograd.Variable(tensor.type(dtype), requires_grad=False)
    else:
        return autograd.Variable(tensor.type(dtype), requires_grad=False, volatile=True) # Solves GPU running out of memory issue
    
def data_from_df_to_list(df):
    tuples = [tuple(x) for x in df.values]
    data = [(r, s.split(), l.split()) for r, s, l, _ in tuples]
    return data

In [351]:
# def neg_log_likelihood(self, sentence, tags):
#     feats = self._get_lstm_features(sentence)
#     forward_score = self._forward_alg(feats)
#     gold_score = self._score_sentence(feats, tags)
#     return forward_score - gold_score

# neg_log_likelihood(model, test_sent, test_tags) # Works outside!!!

In [357]:
features = model._get_lstm_features(test_sent) # Works now for GPU
score = model._score_sentence(features, test_tags) # Works now for GPU
alpha = model._forward_alg(features) # Works now for GPU
path_score, best_path = model._viterbi_decode(features) # Works now for GPU

# model.neg_log_likelihood(features, test_tags) # AssertionError: Embedding doesn't compute the gradient w.r.t. the indices